# BLOOM


## Cargar y Configurar el Modelo

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Cargar el tokenizador y el modelo
tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')
model = AutoModelForSequenceClassification.from_pretrained('microsoft/MiniLM-L12-H384-uncased', num_labels=2)


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

C:\Users\micky\.conda\envs\InteligenciArtificial\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\micky\.cache\huggingface\hub\models--microsoft--MiniLM-L12-H384-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\micky\.conda\envs\InteligenciArtificial\Lib\site-packages\huggingface_hub

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Cargar y Preprocesar el Dataset

In [3]:
# Cargar el dataset
dataset = load_dataset('imdb')

# Tokenizar los textos
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
small_train_dataset = tokenized_datasets['train'].shuffle(seed=42).select([i for i in range(500)])  # reducir el tamaño
small_test_dataset = tokenized_datasets['test'].shuffle(seed=42).select([i for i in range(500)])   # reducir el tamaño


## Configurar el Entrenamiento

In [7]:
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              # número de épocas
    per_device_train_batch_size=16,   
    per_device_eval_batch_size=16,    
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

## Entrenamiento y Evaluacion

In [ ]:
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=small_train_dataset,   
    eval_dataset=small_test_dataset      
)

trainer.train()


In [ ]:
eval_result = trainer.evaluate()

print(f"Evaluation results: {eval_result}")


In [ ]:
model.save_pretrained('./finetuned_model')
tokenizer.save_pretrained('./finetuned_model')
